# Combine Monthly EVI data from NASA's LP DAAC into a single dataset

Temporal support: monthly, 2015--2020 <br>
Spaital support: 0.05-deg, global

https://doi.org/10.5067/MODIS/MOD13C2.006 <br>
https://lpdaac.usgs.gov/

NOTE: Use `geopy` conda environment.


In [1]:
import sys
sys.path.insert(0, "../src")

import xarray as xr
import rioxarray # needed to load rasterio backend
import data_utils

In [2]:
paths_evi = "../data/exp_raw/*.hdf"
ds = xr.open_mfdataset(
    paths_evi, 
    engine="rasterio", 
    variable="CMG 0.05 Deg Monthly EVI",
    mask_and_scale=True, 
    preprocess=data_utils.prep_evi,
    combine="nested",
    concat_dim="time",
    parallel=True
    )
ds

/home/jj829/.conda/envs/geopy/lib/python3.9/site-packages/rasterio/__init__.py:220: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
/home/jj829/.conda/envs/geopy/lib/python3.9/site-packages/rasterio/__init__.py:220: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


<xarray.Dataset>
Dimensions:  (time: 72, lon: 1400, lat: 880)
Coordinates:
  * lon      (lon) float64 -130.0 -129.9 -129.9 -129.8 ... -60.12 -60.07 -60.02
  * lat      (lat) float64 61.97 61.92 61.88 61.83 ... 18.17 18.12 18.08 18.02
  * time     (time) datetime64[ns] 2015-01-01 2015-02-01 ... 2020-12-01
Data variables:
    evi      (time, lon, lat) float32 2.5e+06 1.4e+06 7.7e+06 ... nan nan nan

In [3]:
ds.to_netcdf("../data/exp_pro/MODIS_EVI_conus.nc", format="NETCDF4", engine="h5netcdf")
ds.close()